In [1]:
from test import load_data_sparse
from SGD import *
from ALS import *
from surprise_models import *
from surprise.model_selection import train_test_split

In [2]:
def calculate_rmse(real_labels, predictions):
    """Calculate RMSE."""
    return np.linalg.norm(real_labels - predictions) / np.sqrt(len(real_labels))

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"
test_name = "data/sampleSubmission.csv"

Load trainset:

In [4]:
sp_ratings, pd_ratings = load_data_sparse(data_name, False)

Convert it into surprise

In [5]:
spr_data = pandas_to_surprise(pd_ratings)

In [11]:
spr_data.df[0:5]

,User,Movie,Rating
0,43,0,4
1,60,0,3
2,66,0,4
3,71,0,3
4,85,0,5


Split into test and dataset

In [6]:
trainset, testset = train_test_split(spr_data, test_size=.2)

build prediction set

In [7]:
_, pd_pred = load_data_sparse(test_name, False)

In [ ]:
pd_pred

convert it into surprise:

In [8]:
pred = pandas_to_surprise(pd_pred, True)

In [10]:
svd_test, svd_final, rmse = surprise_SVD(trainset, testset, pred)

RMSE: 1.0080


In [11]:
so_test, so_final, so_rmse = surprise_slopeOne(trainset, testset, pred)

RMSE: 1.0019


In [9]:
num_features = 40  # K in the lecture notes
lambda_user = 0.1
lambda_film = 0.1
stop_criterion = 1e-3

In [10]:
als_test, als_final = ALS_CV(trainset, testset, pred, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using ALS...


In [ ]:
sgd_test, sgd_final = matrix_factorization_SGD_CV(trainset, testset, pred, num_features, lambda_user, lambda_film, stop_criterion)

In [24]:
_,_,rat = get_testset_indices(testset)

In [28]:
calculate_rmse(rat, preds)

1.0271070183303297

In [15]:
def create_submission_from_prediction(prediction, output_name):

    def prediction_transformed(prediction, ids_process):
        """ return the prediction transformed for the submission """
        y = []
        for i in range(len(ids_process)):
            row = ids_process[i][0]
            col = ids_process[i][1]
            y.append(prediction[row - 1, col - 1])
        return y

    def create_csv_submission(ids, y_pred, name):
        """
        Creates an output file in csv format for submission to kaggle
        Arguments: ids (event ids associated with each prediction)
                   y_pred (predicted class labels)
                   name (string name of .csv output file to be created)
        """
        with open(name, 'w') as csvfile:
            fieldnames = ['Id', 'Prediction']
            writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
            writer.writeheader()
            for r1, r2 in zip(ids, y_pred):
                writer.writerow({'Id': str(r1), 'Prediction': float(r2)})

    def round(x):
        if (x < 1):
            return 1
        elif (x > 5):
            return 5
        else:
            return x

    def transform(ids_txt):
        """ split the text index"""

        def deal_line(line):
            pos, rating = line.split(',')
            return str(pos)

        ids = [deal_line(line) for line in ids_txt]
        return ids

    prediction = np.vectorize(round)(prediction)

    DATA_TEST_PATH = 'data/sampleSubmission.csv'

    ids_txt = read_txt(DATA_TEST_PATH)[1:]
    ids_process = [deal_line(line) for line in ids_txt]

    # prediction under the right format
    # y = prediction_transformed(prediction, ids_process)
    y = np.vectorize(round)(prediction)
    
    y = np.rint(y)

    ids = transform(ids_txt)
    OUTPUT_PATH = output_name
    create_csv_submission(ids, y, OUTPUT_PATH)

In [14]:
als_final.shape

(1176952,)

In [16]:
create_submission_from_prediction(als_final, 'svdfinal.csv')